In [1]:
import os
import json
from datetime import datetime

In [14]:
# idx_path_summar = "../IDX_Channel/data/specic"
idx_path_detail = "/Users/albert/Documents/Finances/data/raw/alternative_data/news/idx_channel/detailed"
kabar_bursa_detail = "/Users/albert/Documents/Finances/data/raw/alternative_data/news/kabar_bursa/kabarbursa_detailed"

output_file = "/Users/albert/Documents/Finances/data/processed/20250901_20260131/merged_detailed_20250901-20260131.json"

In [15]:
!ls {idx_path_detail}

!ls {kabar_bursa_detail}

# !ls {idx_path_summar}

idxchannel_market_news_summary20260101_20260116_161822_DETAILED_20260116_163310.json
idxchannel_market_news_summary20260115_20260119_001653_DETAILED_20260120_215755.json
idxchannel_market_news_summary20260119_20260120_212502_DETAILED_20260120_215709.json
idxchannel_market_news_summary20260121_20260131_220314_DETAILED_20260131_220730.json
idxchannel_market_news_summary20260128_20260129_165807_DETAILED_20260129_165925.json
ticker
kabarbursa_20250901_20251231_154146_DETAILED_20260116_164649.json
kabarbursa_20260101_20260116_161739_DETAILED_20260116_162649.json
kabarbursa_20260116_20260120_212610_DETAILED_20260120_215106.json
kabarbursa_20260121_20260131_220138_DETAILED_20260131_220353.json
kabarbursa_20260129_20260129_170207_DETAILED_20260129_170535.json


In [16]:
def load_json_files_from_dir(directory):
    all_data = []

    if not os.path.exists(directory):
        print(f"❌ Folder tidak ditemukan: {directory}")
        return all_data

    files = [f for f in os.listdir(directory) if f.endswith(".json")]

    print(f"📂 Membaca {len(files)} file dari {directory}")

    for file in files:
        file_path = os.path.join(directory, file)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                if isinstance(data, list):
                    all_data.extend(data)
                else:
                    print(f"⚠️ Bukan list JSON: {file}")
        except Exception as e:
            print(f"❌ Gagal membaca {file}: {e}")

    return all_data

In [17]:
print("🚀 Menggabungkan dataset...")

idx_data = load_json_files_from_dir(idx_path_detail)
# idx_summ_data = load_json_files_from_dir(idx_path_summar)
kabar_bursa_data = load_json_files_from_dir(kabar_bursa_detail)

# merged_data = idx_data + idx_path_summar + kabar_bursa_data
merged_data = idx_data + kabar_bursa_data

print(f"📊 Total artikel sebelum sorting: {len(merged_data)}")


def safe_date(item):
    try:
        return datetime.strptime(item.get("date", "1900-01-01"), "%Y-%m-%d")
    except:
        return datetime(1900, 1, 1)

def filter_fields(item):
    """Keep only required fields"""
    return {
        "title": item.get("title", ""),
        "date": item.get("date", ""),
        "source": item.get("source", ""),
        "full_content": item.get("full_content", ""),
        "summary": item.get("summary", "")
    }

merged_data.sort(key=safe_date)

filtered_data = [filter_fields(item) for item in merged_data]

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, indent=2, ensure_ascii=False)

print("✅ SELESAI")
print(f"📁 File hasil: {output_file}")
print(f"📊 Total artikel akhir: {len(merged_data)}")


🚀 Menggabungkan dataset...
📂 Membaca 5 file dari /Users/albert/Documents/Finances/data/raw/alternative_data/news/idx_channel/detailed
📂 Membaca 5 file dari /Users/albert/Documents/Finances/data/raw/alternative_data/news/kabar_bursa/kabarbursa_detailed
📊 Total artikel sebelum sorting: 3304
✅ SELESAI
📁 File hasil: /Users/albert/Documents/Finances/data/processed/20250901_20260131/merged_detailed_20250901-20260131.json
📊 Total artikel akhir: 3304


In [6]:
import json
import os
from datetime import datetime

# SOURCE FILE PATHS
idx_json_path = "/Users/albert/Documents/Finances/data/raw/alternative_data/news/idx_channel/detailed/idxchannel_market_news_summary20260121_20260131_220314_DETAILED_20260131_220730.json"

kb_json_path = "/Users/albert/Documents/Finances/data/raw/alternative_data/news/kabar_bursa/kabarbursa_detailed/kabarbursa_20260121_20260131_220138_DETAILED_20260131_220353.json"

# GOAL OUTPUT PATH
goal_path = "/Users/albert/Documents/Finances/data/processed/20260121_20260131"
os.makedirs(goal_path, exist_ok=True)

output_file = os.path.join(
    goal_path,
    "merged_news_filtered_20260121_20260131.json"
)

# HELPER FUNCTIONS
def load_json_file(file_path):
    """Load single JSON file safely"""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise ValueError(f"File {file_path} tidak berisi list JSON")
    return data


def safe_date(item):
    """Safe date parsing for sorting"""
    date_str = item.get("date")
    if not date_str:
        return datetime(1900, 1, 1)
    try:
        return datetime.fromisoformat(date_str[:10])
    except (ValueError, TypeError):
        return datetime(1900, 1, 1)


def filter_fields(item):
    """Keep only required fields"""
    return {
        "title": item.get("title", ""),
        "date": item.get("date", ""),
        "source": item.get("source", ""),
        "full_content": item.get("full_content", ""),
        "summary": item.get("summary", "")
    }

# LOAD DATA
idx_data = load_json_file(idx_json_path)
kabar_bursa_data = load_json_file(kb_json_path)

print(f"📄 IDX articles        : {len(idx_data)}")
print(f"📄 Kabar Bursa articles: {len(kabar_bursa_data)}")

# MERGE & SORT
merged_data = idx_data + kabar_bursa_data
merged_data.sort(key=safe_date)

print(f"📊 Total artikel setelah merge: {len(merged_data)}")

# FILTER REQUIRED FIELDS
filtered_data = [filter_fields(item) for item in merged_data]

# SAVE OUTPUT
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, indent=2, ensure_ascii=False)

print("✅ SELESAI")
print(f"📁 File hasil : {output_file}")
print(f"📊 Total final: {len(filtered_data)}")


📄 IDX articles        : 393
📄 Kabar Bursa articles: 210
📊 Total artikel setelah merge: 603
✅ SELESAI
📁 File hasil : /Users/albert/Documents/Finances/data/processed/20260121_20260131/merged_news_filtered_20260121_20260131.json
📊 Total final: 603
